In [3]:
import pandas as pd

In [63]:
WASTE = 'waste (tons annual)'
RECYCLING = 'recycling (tons annual)'
POPULATION = 'population'

DF_data = {2014: {WASTE: 5030, RECYCLING: 1270, POPULATION: 11001},
           2019: {WASTE: 4930, RECYCLING: 1764, POPULATION: 11027},
           2020: {WASTE: 5042, RECYCLING: 2101, POPULATION: 11541}}
DF_data = pd.DataFrame(DF_data).transpose().reset_index().rename({'index': 'year'}, axis='columns')

In [64]:
LB_PER_TON = 2000.0

RECYCLING_RATE = 'recycling rate'
WASTE_PPD = 'waste (pounds per person per day)'

DF_data[RECYCLING_RATE] = DF_data[RECYCLING] / (DF_data[RECYCLING] + DF_data[WASTE])
DF_data[WASTE_PPD] = DF_data[WASTE] / DF_data[POPULATION] / 365.0 * LB_PER_TON

In [65]:
WASTE_2014 = DF_data.set_index('year').loc[2014, WASTE]
WASTE_TARGET_2020 = WASTE_2014 * 0.8
WASTE_CHANGE_PER_YEAR = (WASTE_TARGET_2020 - WASTE_2014) / 6.0

WASTE_TARGET = 'waste target (tons annual)'

DF_data[WASTE_TARGET] = (DF_data['year'] - 2014) * WASTE_CHANGE_PER_YEAR + WASTE_2014

In [66]:
DF_data

,year,waste (tons annual),recycling (tons annual),population,recycling rate,waste (pounds per person per day),waste target (tons annual)
0,2014,5030,1270,11001,0.201587,2.505376,5030.000000
1,2019,4930,1764,11027,0.263520,2.449778,4191.666667
2,2020,5042,2101,11541,0.294134,2.393848,4024.000000


In [67]:
import altair as alt

In [90]:
def plot_horizontal_with_label(yval, scale_domain, label, color='green'):
    horiz_line = alt.Chart().mark_rule(color=color, size=2).encode(y=alt.datum(yval))
    text_mark = alt.Chart().encode(y=alt.datum(yval),
                                   x=alt.XDatum(scale_domain[1], scale=alt.Scale(domain=scale_domain)),
                                   text=alt.datum(label)).mark_text(baseline='bottom', align='right')
    return horiz_line + text_mark

## Waste totals

In [68]:
waste_line = alt.Chart(DF_data).mark_line(color='red', point=alt.OverlayMarkDef(color="red")).encode(alt.X('year:Q', scale=alt.Scale(domain=[2014, 2020]), axis=alt.Axis(format=".0f")), y=WASTE)
recycle_line = alt.Chart(DF_data).mark_line(point=True).encode(alt.X('year:Q', scale=alt.Scale(domain=[2014, 2020]), axis=alt.Axis(format=".0f")), y=RECYCLING)
goal_line = alt.Chart(DF_data).mark_line(strokeDash=[12, 6], size=2, color='green').encode(alt.X('year:Q', scale=alt.Scale(domain=[2014, 2020]), axis=alt.Axis(format=".0f")), y=WASTE_TARGET)
(waste_line + recycle_line + goal_line).properties(
    title="Dobbs Ferry recycling and waste totals vs targets",
    width=600,
    height=360,
)

alt.LayerChart(...)

## Rate vs average

> In 2020, the County continued to demonstrate that it’s a regional leader in the field of environmental management by posting a recycling rate of 51.36%, far outpacing the EPA national average of 32.1%.

In [92]:
rate_line = alt.Chart(DF_data).mark_line(color='blue', point=alt.OverlayMarkDef(color="blue")).encode(alt.X('year:Q', scale=alt.Scale(domain=[2014, 2020]), axis=alt.Axis(format=".0f")), y=RECYCLING_RATE)
goal_line = plot_horizontal_with_label(0.5136, [2014, 2020], "Westchester recycling rate in 2020")
goal_line2 = plot_horizontal_with_label(0.321, [2014, 2020], "National recycling rate in 2020")
(goal_line + rate_line + goal_line2).properties(
    title="Dobbs Ferry recycling rate vs target",
    width=600,
    height=360,
)

alt.LayerChart(...)

## PPD

> The State’s Plan, “Beyond Waste,” challenged Planning Units to drastically reduce solid waste disposed (i.e., non-recyclable waste) to 0.6 pounds per person per day (“PPD”) by 2030. Westchester is working very hard to meet the State’s goal. In 2020, the County’s average of 3.7 PPD outperformed the New York State average of 4.5 PPD, as well as the national average of 4.9 PPD (2018 rate).

In [93]:
ppd_line = alt.Chart(DF_data).mark_line(color='red', point=alt.OverlayMarkDef(color="red")).encode(alt.X('year:Q', scale=alt.Scale(domain=[2014, 2020]), axis=alt.Axis(format=".0f")), y=WASTE_PPD)


goal_lines = plot_horizontal_with_label(3.7, [2014, 2020], "County rate in 2020")
goal_lines = goal_lines + plot_horizontal_with_label(0.6, [2014, 2020], "NYS goal for 2030")
(goal_lines + ppd_line).properties(
    title="Dobbs Ferry waste per person vs county average",
    width=600,
    height=360,
)

alt.LayerChart(...)